In [0]:
df = spark.read.json("abfss://gizmobox@stgdeltadatabricks.dfs.core.windows.net/landing/operational/customers/*.json")
df.display()

In [0]:
from pyspark.sql.functions import col,lit,max

In [0]:
df2 = df.where("customer_id is not null") \
        .distinct()

In [0]:
df3 = df2.groupBy(col("customer_id")) \
         .agg(max("created_timestamp").alias("created_timestamp"))
df4 = df3.alias("d1").join(df2.alias("d2")) \
                     .where((col("d1.customer_id") == col("d2.customer_id")) & (col("d1.created_timestamp") == col("d2.created_timestamp"))) \
                     .select(col("d2.created_timestamp").cast("timestamp"),col("d2.customer_id"),col("d2.customer_name"),col("d2.email"),col("d2.date_of_birth").cast("date"),col("d2.member_since").cast("date"),col("d2.telephone"))

df4.display()

   

In [0]:
%sql
drop table gizmobox.silver.customers

In [0]:
df4.write.format("delta").saveAsTable("gizmobox.silver.customers")

In [0]:
df = spark.table("gizmobox.silver.customers")
df.display()

In [0]:
%sql
select * from gizmobox.silver.customers